In [ ]:
# install required libraries
!pip3 install sentence-transformers

     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 2.0MB 24.0MB/s 
     |████████████████████████████████| 1.2MB 41.7MB/s 
     |████████████████████████████████| 3.2MB 53.8MB/s 
     |████████████████████████████████| 870kB 52.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=2f444a7eac4bbc0253f7c8a0d046960542d44279601dded700077788291a7591
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=384237782ba656f2fa1e4e1d71909115660f0ead952e141ebd9eaba880a3b10b
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sentence-transformers sacremoses


In [ ]:
!pip3 install elasticsearch

     |████████████████████████████████| 337kB 18.5MB/s 


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/BERT-FAQ

/content/drive/MyDrive/BERT-FAQ


In [ ]:
!ls

data			     __init__.py  reranker.py
evaluation.py		     metric.py	  searcher.py
faq_bert_finetuning.py	     notebook	  shared
faq_bert.py		     output	  training_data_generator.py
faq_bert_ranker.py	     parser	  webserver.py
hard_negatives_generator.py  __pycache__
indexer.py		     README.md


In [ ]:
# import required dependencies
from evaluation import get_relevance_label_df
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs
from reranker import ReRanker

In [ ]:
output_path="data/CovidFAQ/rank_results"

# load user_query ES results from json files
es_output_path = output_path + "/unsupervised"
es_query_by_question = load_from_json(es_output_path + '/es_query_by_question.json')
es_query_by_answer = load_from_json(es_output_path + '/es_query_by_answer.json')
es_query_by_question_answer = load_from_json(es_output_path + '/es_query_by_question_answer.json')
es_query_by_question_answer_concat = load_from_json(es_output_path + '/es_query_by_question_answer_concat.json')

In [ ]:
# load test_queries, relevance_label_df for ReRanker
query_answer_pair_filepath = 'data/CovidFAQ/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
test_queries = relevance_label_df[relevance_label_df['query_type'] == 'user_query'].question.unique()

In [ ]:
test_queries[:10]

array(['What does Corona and Covid mean?',
       'when will the social distancing end? and what is the economic consequent of the pandemic?',
       'How do I go grocery shopping? ', 'what is covid 19',
       "What is COVID-19's definition?", 'Should I stay home ?',
       'Should I practice social distancing? ', 'How did it started',
       'what are the 3 symptoms of covid-19', 'Should I wear a mask?'],
      dtype=object)

In [ ]:
# total number of test queries
len(test_queries)

1078

**1. Generating BERT prediction results from Answer (BERT-Q-a)"**

In [ ]:
# define rank_field parameter
rank_field="BERT-Q-a"

**query_type="user_query"; neg_type="hard"; loss_type='triplet'**

In [ ]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-03 09:09:49 - Generating BERT top-k results ...
2021-04-03 09:09:50 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-03 09:09:50 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-03 09:10:02 - Use pytorch device: cuda
100%|██████████| 1078/1078 [04:47<00:00,  3.75it/s]
2021-04-03 09:14:50 - Generating BERT top-k results ...
2021-04-03 09:14:50 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-03 09:14:50 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-03 09:14:51 - Use pytorch device: cuda
100%|██████████| 1078/1078 [08:20<00:00,  2.15it/s]
2021-04-03 09:23:13 - Generating BERT top-k results ...
2021-04-03 09:23:13 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-03 09:23:13 - Load SentenceTransformer from folder: output/CovidFAQ/models

**query_type="user_query"; neg_type="simple"; loss_type='triplet'**

In [ ]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-03 09:41:06 - Generating BERT top-k results ...
2021-04-03 09:41:06 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-03 09:41:06 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-03 09:41:16 - Use pytorch device: cuda
100%|██████████| 1078/1078 [04:43<00:00,  3.80it/s]
2021-04-03 09:46:00 - Generating BERT top-k results ...
2021-04-03 09:46:00 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-03 09:46:00 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-03 09:46:01 - Use pytorch device: cuda
100%|██████████| 1078/1078 [08:21<00:00,  2.15it/s]
2021-04-03 09:54:24 - Generating BERT top-k results ...
2021-04-03 09:54:24 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-03 09:54:24 - Load SentenceTransformer from folder: output/Covid

**query_type="faq"; neg_type="hard"; loss_type='triplet'**

In [ ]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-03 10:12:17 - Generating BERT top-k results ...
2021-04-03 10:12:17 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-03 10:12:17 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-03 10:12:29 - Use pytorch device: cuda
100%|██████████| 1078/1078 [04:40<00:00,  3.84it/s]
2021-04-03 10:17:10 - Generating BERT top-k results ...
2021-04-03 10:17:10 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-03 10:17:10 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-03 10:17:11 - Use pytorch device: cuda
100%|██████████| 1078/1078 [08:18<00:00,  2.16it/s]
2021-04-03 10:25:31 - Generating BERT top-k results ...
2021-04-03 10:25:31 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-03 10:25:31 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-03 10

**query_type="faq"; neg_type="simple"; loss_type='triplet'**

In [ ]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-03 10:43:19 - Generating BERT top-k results ...
2021-04-03 10:43:19 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-03 10:43:19 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-03 10:43:29 - Use pytorch device: cuda
100%|██████████| 1078/1078 [04:40<00:00,  3.84it/s]
2021-04-03 10:48:10 - Generating BERT top-k results ...
2021-04-03 10:48:10 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-03 10:48:10 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-03 10:48:12 - Use pytorch device: cuda
100%|██████████| 1078/1078 [08:18<00:00,  2.16it/s]
2021-04-03 10:56:32 - Generating BERT top-k results ...
2021-04-03 10:56:32 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-03 10:56:32 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_faq_1.1
2

**query_type="user_query"; neg_type="hard"; loss_type='softmax'**

In [ ]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-03 11:14:23 - Generating BERT top-k results ...
2021-04-03 11:14:33 - Use pytorch device: cuda
100%|██████████| 1078/1078 [04:08<00:00,  4.34it/s]
2021-04-03 11:18:42 - Generating BERT top-k results ...
2021-04-03 11:18:44 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:20<00:00,  2.45it/s]
2021-04-03 11:26:05 - Generating BERT top-k results ...
2021-04-03 11:26:07 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:49<00:00,  2.30it/s]
2021-04-03 11:33:58 - Generating BERT top-k results ...
2021-04-03 11:34:00 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:49<00:00,  2.30it/s]


**query_type="user_query"; neg_type="simple"; loss_type='softmax'**

In [ ]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-03 11:41:51 - Generating BERT top-k results ...
2021-04-03 11:42:04 - Use pytorch device: cuda
100%|██████████| 1078/1078 [04:08<00:00,  4.33it/s]
2021-04-03 11:46:13 - Generating BERT top-k results ...
2021-04-03 11:46:15 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:21<00:00,  2.44it/s]
2021-04-03 11:53:38 - Generating BERT top-k results ...
2021-04-03 11:53:40 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:49<00:00,  2.30it/s]
2021-04-03 12:01:31 - Generating BERT top-k results ...
2021-04-03 12:01:33 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:49<00:00,  2.30it/s]


**query_type="faq"; neg_type="hard"; loss_type='softmax'**

In [ ]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-03 12:09:24 - Generating BERT top-k results ...
2021-04-03 12:09:35 - Use pytorch device: cuda
100%|██████████| 1078/1078 [04:07<00:00,  4.36it/s]
2021-04-03 12:13:42 - Generating BERT top-k results ...
2021-04-03 12:13:44 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:20<00:00,  2.45it/s]
2021-04-03 12:21:06 - Generating BERT top-k results ...
2021-04-03 12:21:08 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:49<00:00,  2.29it/s]
2021-04-03 12:28:59 - Generating BERT top-k results ...
2021-04-03 12:29:01 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:50<00:00,  2.29it/s]


**query_type="faq"; neg_type="simple"; loss_type='softmax'**

In [ ]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 04:47:10 - Generating BERT top-k results ...
2021-04-04 04:47:23 - Use pytorch device: cuda
100%|██████████| 1078/1078 [04:14<00:00,  4.23it/s]
2021-04-04 04:51:38 - Generating BERT top-k results ...
2021-04-04 04:51:40 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:16<00:00,  2.47it/s]
2021-04-04 04:58:58 - Generating BERT top-k results ...
2021-04-04 04:59:00 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:44<00:00,  2.32it/s]
2021-04-04 05:06:46 - Generating BERT top-k results ...
2021-04-04 05:06:47 - Use pytorch device: cuda
100%|██████████| 1078/1078 [07:50<00:00,  2.29it/s]


**2. Generating BERT prediction results from Question (BERT-Q-q)"**


In [ ]:
# define rank_field parameter
rank_field="BERT-Q-q"

**query_type="user_query"; neg_type="hard"; loss_type='triplet'**

In [ ]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 05:14:40 - Generating BERT top-k results ...
2021-04-04 05:14:40 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-04 05:14:40 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-04 05:14:52 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:54<00:00,  4.59it/s]
2021-04-04 05:18:47 - Generating BERT top-k results ...
2021-04-04 05:18:47 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-04 05:18:47 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-04 05:18:49 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:48<00:00,  2.64it/s]
2021-04-04 05:25:39 - Generating BERT top-k results ...
2021-04-04 05:25:39 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_user_query_1.1
2021-04-04 05:25:39 - Load SentenceTransformer from folder: output/CovidFAQ/models

**query_type="user_query"; neg_type="simple"; loss_type='triplet'**

In [ ]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 05:40:24 - Generating BERT top-k results ...
2021-04-04 05:40:24 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-04 05:40:24 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-04 05:40:37 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:57<00:00,  4.54it/s]
2021-04-04 05:44:35 - Generating BERT top-k results ...
2021-04-04 05:44:35 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-04 05:44:35 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-04 05:44:36 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:49<00:00,  2.63it/s]
2021-04-04 05:51:26 - Generating BERT top-k results ...
2021-04-04 05:51:26 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_user_query_1.1
2021-04-04 05:51:26 - Load SentenceTransformer from folder: output/Covid

**query_type="faq"; neg_type="hard"; loss_type='triplet'**

In [ ]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 06:06:18 - Generating BERT top-k results ...
2021-04-04 06:06:18 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-04 06:06:18 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-04 06:06:30 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:56<00:00,  4.55it/s]
2021-04-04 06:10:27 - Generating BERT top-k results ...
2021-04-04 06:10:27 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-04 06:10:27 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-04 06:10:28 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:52<00:00,  2.61it/s]
2021-04-04 06:17:22 - Generating BERT top-k results ...
2021-04-04 06:17:22 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-04 06:17:22 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_hard_faq_1.1
2021-04-04 06

**query_type="faq"; neg_type="simple"; loss_type='triplet'**

In [ ]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 06:32:14 - Generating BERT top-k results ...
2021-04-04 06:32:14 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-04 06:32:14 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-04 06:32:25 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:56<00:00,  4.55it/s]
2021-04-04 06:36:22 - Generating BERT top-k results ...
2021-04-04 06:36:22 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-04 06:36:22 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-04 06:36:24 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:50<00:00,  2.63it/s]
2021-04-04 06:43:15 - Generating BERT top-k results ...
2021-04-04 06:43:15 - Load pretrained SentenceTransformer: output/CovidFAQ/models/triplet_simple_faq_1.1
2021-04-04 06:43:15 - Load SentenceTransformer from folder: output/CovidFAQ/models/triplet_simple_faq_1.1
2

**query_type="user_query"; neg_type="hard"; loss_type='softmax'**

In [ ]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 06:58:02 - Generating BERT top-k results ...
2021-04-04 06:58:12 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:43<00:00,  4.83it/s]
2021-04-04 07:01:56 - Generating BERT top-k results ...
2021-04-04 07:01:58 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:26<00:00,  2.79it/s]
2021-04-04 07:08:26 - Generating BERT top-k results ...
2021-04-04 07:08:27 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:54<00:00,  2.60it/s]
2021-04-04 07:15:23 - Generating BERT top-k results ...
2021-04-04 07:15:25 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:53<00:00,  2.61it/s]


**query_type="user_query"; neg_type="simple"; loss_type='softmax'**

In [ ]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 07:22:20 - Generating BERT top-k results ...
2021-04-04 07:22:31 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:43<00:00,  4.83it/s]
2021-04-04 07:26:15 - Generating BERT top-k results ...
2021-04-04 07:26:17 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:21<00:00,  2.83it/s]
2021-04-04 07:32:39 - Generating BERT top-k results ...
2021-04-04 07:32:41 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:55<00:00,  2.60it/s]
2021-04-04 07:39:38 - Generating BERT top-k results ...
2021-04-04 07:39:40 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:53<00:00,  2.61it/s]


**query_type="faq"; neg_type="hard"; loss_type='softmax'**

In [ ]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 07:46:34 - Generating BERT top-k results ...
2021-04-04 07:46:46 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:42<00:00,  4.84it/s]
2021-04-04 07:50:29 - Generating BERT top-k results ...
2021-04-04 07:50:31 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:25<00:00,  2.80it/s]
2021-04-04 07:56:57 - Generating BERT top-k results ...
2021-04-04 07:56:59 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:53<00:00,  2.60it/s]
2021-04-04 08:03:54 - Generating BERT top-k results ...
2021-04-04 08:03:56 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:58<00:00,  2.58it/s]


**query_type="faq"; neg_type="simple"; loss_type='softmax'**

In [ ]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/CovidFAQ/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [ ]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df,
    rank_field=rank_field
)

In [ ]:
# generate directory structure
pred_output_path = output_path + "/supervised/" + rank_field + "/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(pred_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, pred_output_path + '/bert_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, pred_output_path + '/bert_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, pred_output_path + '/bert_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, pred_output_path + '/bert_query_by_question_answer_concat.json')

2021-04-04 08:10:56 - Generating BERT top-k results ...
2021-04-04 08:10:58 - Use pytorch device: cuda
100%|██████████| 1078/1078 [03:42<00:00,  4.84it/s]
2021-04-04 08:14:41 - Generating BERT top-k results ...
2021-04-04 08:14:43 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:25<00:00,  2.79it/s]
2021-04-04 08:21:10 - Generating BERT top-k results ...
2021-04-04 08:21:12 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:54<00:00,  2.60it/s]
2021-04-04 08:28:08 - Generating BERT top-k results ...
2021-04-04 08:28:10 - Use pytorch device: cuda
100%|██████████| 1078/1078 [06:52<00:00,  2.61it/s]
